## Module loading

In [2]:
from importlib import reload

import warnings
warnings.filterwarnings("ignore")

import torch
import torch.optim as optim
import torch.nn as nn
import numpy as np

from lib.data import tools
reload(tools)
from lib.data.tools import get_dataloader
from lib.train.runners import PytorchRunner
from lib.train.metrics import IoUMetricMeter
from models.modelInterface import BDD100kModel
import lib.simulation.env
reload(lib.simulation.env)
from lib.simulation.env import (get_image_paths, get_label_paths, get_transforms)

## Set parameters

In [3]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
pkg_name = "10k" # 100k or 10k
output_size = (512,1024)
task_name = "sem_seg" # "drivable", "sem_seg"
config_file = "/home/zekun/drivable/src/models/sem_seg/config-deeplabv3plus-sem_seg.py"
checkpoint_file = "/home/zekun/drivable/outputs/semantic/db/models/deeplabv3+_r50-d8_512x1024_80k_sem_seg_bdd100k.pth"
new_checkpoint_file = "/home/zekun/drivable/outputs/semantic/db/models/deeplabv3+_backbone-test.pth"
train_attr_file = f"/home/zekun/drivable/data/bdd100k/labels/{pkg_name}/bdd100k_labels_images_attributes_train.json"
val_attr_file = f"/home/zekun/drivable/data/bdd100k/labels/{pkg_name}/bdd100k_labels_images_attributes_val.json"
output_dir = "/home/zekun/drivable/outputs/semantic"

batchsize = 4
learn_rate = 1e-4
epochs = 10
num_workers = 0

In [4]:
IMAGE_PATH, IMAGE_PATH_TRAIN, IMAGE_PATH_VAL = get_image_paths("/home/zekun/drivable/", pkg_name)
LABEL_PATH, LABEL_PATH_TRAIN, LABEL_PATH_VAL = get_label_paths("/home/zekun/drivable/", task_name, pkg_name)
train_transform = get_transforms(task_name, output_size, "train")
val_transform = get_transforms(task_name, output_size, "test")
classes_num = 19

conditions = []

In [5]:
from lib.data.condparser import BDD100KConditionParser, ConditionParserMode
import json

all_conditions = tools.get_all_appeared_conditions(val_attr_file, ConditionParserMode.VALUE_LIST)

conditions = []
for condition in all_conditions:
    if "snowy" in condition or "rainy" in condition:
        conditions.append(condition)

## Data loaders

In [6]:
train_fns = tools.get_img_paths_by_conditions(conditions, train_attr_file, IMAGE_PATH_TRAIN)
val_fns = tools.get_img_paths_by_conditions(conditions, val_attr_file, IMAGE_PATH_VAL)

train_loader = get_dataloader(
    train_fns,
    batch_size=batchsize,
    workers=num_workers,
    transform=train_transform,
    img_path=IMAGE_PATH_TRAIN,
    lbl_path=LABEL_PATH_TRAIN,
    is_train=True,
)
val_loader = get_dataloader(
    val_fns,
    batch_size=batchsize,
    workers=num_workers,
    transform=val_transform,
    img_path=IMAGE_PATH_VAL,
    lbl_path=LABEL_PATH_VAL,        
    is_train=False,
)

print(len(train_fns))
print(len(val_fns))

846
133


## Model initialization

In [7]:
model = BDD100kModel(backbone=tools.load_mmcv_checkpoint(config_file, checkpoint_file))
# tools.load_checkpoint(model, checkpoint_file)
# tools.save_model(model.backbone, checkpoint_file)

# optimizer = optim.Adam(model.parameters(), lr=learn_rate)
optimizer = optim.SGD(model.parameters(), lr=learn_rate, momentum=0.9, weight_decay=0.0005)
criterion = nn.CrossEntropyLoss(ignore_index=255)

runner = PytorchRunner(optimizer, criterion, train_loader, val_loader, IoUMetricMeter(classes_num), DEVICE, True)

## Model training

In [ ]:
# runner.train(model, epochs)
for i in range(epochs)
    train_log = runner.train(model)
    print(f"Train: loss={train_log['loss']}, pAcc={train_log['pAcc']}, mIoU={train_log['mIoU']}")
    train_log = runner.validate(model)
    print(f"Validate: loss={train_log['loss']}, pAcc={train_log['pAcc']}, mIoU={train_log['mIoU']}")

## Model Validation

In [8]:
valid_logs = runner.validate(model)
print(valid_logs)

Valid: 100%|██████████| 34/34 [00:11<00:00,  2.90it/s, loss=0.31, pAcc=0.919] 

{'loss': 0.31038389058041393, 'pAcc': 0.918601328211798, 'mIoU': 0.6038449005651612, 'Acc': [0.9724636586155133, 0.7772915426521523, 0.8889251096569247, 0.4004800461343038, 0.859280898155459, 0.8028688829214464, 0.8244922993815872, 0.8830381261444189, 0.9225955646178019, 0.6433837160233672, 0.9682988168505482, 0.808794401162023, 0.9224801428483468, 0.9174735886385387, 0.7903437843182594, 0.9080582125002448, 0.0, 0.0], 'IoU': [0.9424369845420645, 0.6475604461791583, 0.8391755246753493, 0.17805028185231425, 0.6177647950261956, 0.5520584683509385, 0.7046915955272334, 0.7154117880337528, 0.8600042310424069, 0.5097171635175117, 0.9323175629856202, 0.6892724147808651, 0.5594055709058323, 0.8837172889121037, 0.5912744708968375, 0.6463496229447152, 0.0, 0.0]}


In [9]:
for k,v in valid_logs.items():
    print(k, "=", v)

loss = 0.31038389058041393
pAcc = 0.918601328211798
mIoU = 0.6038449005651612
Acc = [0.9724636586155133, 0.7772915426521523, 0.8889251096569247, 0.4004800461343038, 0.859280898155459, 0.8028688829214464, 0.8244922993815872, 0.8830381261444189, 0.9225955646178019, 0.6433837160233672, 0.9682988168505482, 0.808794401162023, 0.9224801428483468, 0.9174735886385387, 0.7903437843182594, 0.9080582125002448, 0.0, 0.0]
IoU = [0.9424369845420645, 0.6475604461791583, 0.8391755246753493, 0.17805028185231425, 0.6177647950261956, 0.5520584683509385, 0.7046915955272334, 0.7154117880337528, 0.8600042310424069, 0.5097171635175117, 0.9323175629856202, 0.6892724147808651, 0.5594055709058323, 0.8837172889121037, 0.5912744708968375, 0.6463496229447152, 0.0, 0.0]


In [1]:
tools.save_model(model.backbone, new_checkpoint_file, epochs, valid_logs['mIoU'])

NameError: name 'tools' is not defined

## Appendix

In [64]:
x1 = [1,1,1]
x2 = [1,1,2]

def calculate_iou(pred_mask, true_mask, class_label):
    pred_class = pred_mask == class_label
    true_class = true_mask == class_label

    intersection = np.logical_and(pred_class, true_class)
    union = np.logical_or(pred_class, true_class)

    iou_score = np.sum(intersection) / (np.sum(union) + 0.00005)
    return iou_score

def calculate_mean_iou(pred_mask, true_mask, num_classes):
    miou_sum = 0.0
    for class_label in range(num_classes):
        iou = calculate_iou(pred_mask, true_mask, class_label)
        print(iou)
        miou_sum += iou

    mean_iou = miou_sum / num_classes
    return mean_iou

num_classes = 3
miou = calculate_mean_iou(np.array([x1, x1]), np.array([x2, x1]), num_classes)
print("mIoU:", miou)
print(x1[0])

0.0
0.8333263889467588
0.0
mIoU: 0.2777754629822529
1


In [10]:
# from importlib import reload
# from mmseg.apis import inference_model, init_model, show_result_pyplot
# from mmengine.config import Config
# from mmengine.runner import load_checkpoint
# from collections import defaultdict
# import numpy as np
# import matplotlib.pyplot as plt
# import torch

# from PIL import Image
# from bdd100k.label.palette import get_palette
# import lib.simulation.env
# reload(lib.simulation.env)
# from lib.simulation.env import get_transforms
# import models.modelInterface
# reload(models.modelInterface)
# from models.modelInterface import BDD100kModel
# from lib.data.tools import load_mmcv_checkpoint

# config_file = "/home/zekun/drivable/src/models/sem_seg/config-deeplabv3plus-sem_seg.py"
# checkpoint_file = "/home/zekun/drivable/outputs/semantic/db/models/deeplabv3+_r50-d8_512x1024_80k_sem_seg_bdd100k.pth"
# image_name = "../data/bdd100k/images/10k/val/7d06fefd-f7be05a6.jpg"
# label_name = "../data/bdd100k/labels/10k/sem_seg/masks/val/7d06fefd-f7be05a6.png"

# model = BDD100kModel(load_mmcv_checkpoint(config_file, checkpoint_file))
# model.eval()

# output_size = (512,1024)

# transform = get_transforms("sem_seg", output_size, "train")

# data = transform(Image.open(image_name), Image.open(label_name))

# print(data[0].shape[-2:])

# with torch.no_grad():
#     results = model(data[0].unsqueeze(0))
#     results = torch.argmax(results, dim=1).squeeze().detach().cpu()
# print(results.shape)

# print(data[0].shape, data[1].shape)
# print(data[1])
# # print(np.array(Image.open(image_name)))

# pred_img_np = data[1].numpy()
# print(np.unique(pred_img_np))
# print(len(np.unique(pred_img_np)))
# pred_img_np = (pred_img_np * 255).astype(np.int8)
# pred_img = Image.new('P', (pred_img_np.shape[1], pred_img_np.shape[0]))
# palettes = get_palette("sem_seg")
# pred_img.putpalette(palettes)
# pred_img.putdata(pred_img_np.ravel().tolist())


# plt.figure(figsize=(18, 12))  

# plt.subplot(3,1,1)
# plt.imshow(Image.open(image_name))
# plt.title('Img')

# plt.subplot(3, 1, 2)  
# plt.imshow(data[0].permute(1,2,0).detach().numpy())  
# plt.title('Original Image')


# plt.subplot(3, 1, 3)  
# plt.imshow(pred_img_np)  
# plt.title('Predicted Mask')  

# plt.show()

torch.Size([512, 1024])
torch.Size([512, 1024])
torch.Size([3, 512, 1024]) torch.Size([512, 1024])
tensor([[0.0078, 0.0078, 0.0078,  ..., 0.0392, 0.0392, 0.0392],
        [0.0078, 0.0078, 0.0078,  ..., 0.0392, 0.0392, 0.0392],
        [0.0078, 0.0078, 0.0078,  ..., 0.0392, 0.0392, 0.0392],
        ...,
        [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000]])


In [166]:
# from mmengine.dataset import Compose

# cfg = Config.fromfile(config_file)
# transform = Compose(cfg.train_pipeline)

# data = transform(dict(img_path=image_name))

# print(data["inputs"].shape)

# plt.imshow(data["inputs"].permute(1,2,0).detach().numpy()/255)  


torch.Size([3, 512, 1024])
